In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, FastICA, NMF

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [6]:
"""
from jupyterthemes import jtplot
jtplot.style("chesterish")
#jtplot.style("grade3")
"""
plt.rcParams["figure.figsize"] = (10,10)

In [11]:
def rotate_point(origin, point, angle):
    """
        Rotate a point counterclockwise by a given angle around a given origin.
        The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point
    cos = math.cos(angle)
    sin = math.sin(angle)
    x_dif = (px - ox)
    y_dif = (py - oy)
    
    qx = ox + cos * x_dif - sin * y_dif
    qy = oy + sin * x_dif + cos * y_dif

    return qx, qy

def generate_linear_cloud(nb_points, sigma):
    """
        Generate a cloud of point following the y=x equation with a certain value of noise for y.
        Points are generated following the Normal distribution
    """
    x = np.random.randn(1, nb_points)
    noise = np.random.normal(0, sigma, x.shape)
    y = x + noise
    return x, y

def rotate_array(x_ar, y_ar, origin, angle):
    """
        Rotate a 2D array around an origin point counter clockwise by a certain angle in degree.
    """

    x_rot = []
    y_rot = []

    for x, y in zip(x_ar, y_ar):
        x_tmp, y_tmp = rotate_point(origin, (x,y), math.radians(angle))
        x_rot.append(x_tmp)
        y_rot.append(y_tmp)

    return x_rot, y_rot

def plot_samples(S, labels, center, axis_list=None):
    """
        Plot decomposition results. If axis_list is given, vectors of decomposition components
        will be plotted.
        Center can be given to use as the origin of vectors.
    """
    plt.scatter(S[:, 0], S[:, 1], s=2, marker='o', zorder=10,
                c=labels, alpha=0.8)
    if axis_list is not None:
        colors = ['orange', 'red', 'yellow']
        for color, axis in zip(colors, axis_list):
            axis /= axis.std()
            x_axis, y_axis = axis
            # Trick to get legend to work
            plt.plot(0*x_axis+center , 0*y_axis+center, linewidth=1, color=color)
            if color == 'yellow':
                plt.quiver(min(S[:, 0]), min(S[:, 0]), x_axis, y_axis, zorder=11,
                           width=0.01, scale=6, color=color)
            else:
                plt.quiver(center[0], center[1], x_axis, y_axis, zorder=11,
                           width=0.01, scale=6, color=color)
    return

def plot_samples_centered(S, labels, center, axis_list=None):
    """
        Plot decomposition results. If axis_list is given, vectors of decomposition components
        will be plotted.
        Center can be given to use as the origin of vectors.
    """
    plt.scatter(S[:, 0], S[:, 1], s=2, marker='o', zorder=10,
                c=labels, alpha=0.5)
    if axis_list is not None:
        colors = ['orange', 'red', 'yellow']
        for color, axis in zip(colors, axis_list):
            axis /= axis.std()
            x_axis, y_axis = axis
            # Trick to get legend to work
            plt.plot(0*x_axis+center , 0*y_axis+center, linewidth=1, color=color)
            plt.quiver(center[0], center[1], x_axis, y_axis, zorder=11,
                       width=0.01, scale=6, color=color)
            
    return

def Deconvolution_example(point_nb, sigma, angle, centering,
                          ica_algo, ica_tol, ica_non_linearity,
                          nmf_init, nmf_solver, seed):
    
    ### Generate 2 clusters
    if int(seed)!=0:
        np.random.seed(int(seed))
    point_nb = int(point_nb)
    sigma = float(sigma)
    x1, y1 = generate_linear_cloud(point_nb, sigma)
    x2, y2 = generate_linear_cloud(point_nb, sigma)

    ### Rotate one cluster by a certain angle in degree arround an origin point
    origin = (0,0)
    angle = float(angle)
    centering = float(centering)
    x_new, y_new = rotate_array(x2, y2, origin, angle)

    ### Merge clusters into a matrix
    clust_1 = np.vstack((x1, y1)).T
    clust_2 = np.vstack((x_new, y_new)).T
    X = np.concatenate((clust_1, clust_2)) + centering
    labels_clust = np.concatenate((np.repeat("C0",point_nb), np.repeat("C1",point_nb)))
    
    ### Perform PCA, ICA, NMF
    pca = PCA()
    S_pca = pca.fit_transform(X) # Fit PCA

    ica = FastICA(max_iter=10000, tol=float(ica_tol), whiten=True,
                  algorithm=ica_algo, fun=ica_non_linearity)
    S_ica = ica.fit(X).transform(X)  # Fit ICA
    #S_ica = ica.fit_transform(X)
    S_ica /= S_ica.std() # Normalise vectors

    nmf = NMF(max_iter=10000, init=nmf_init, solver=nmf_solver)
    S_nmf = nmf.fit(X).transform(X) # Fit NMF
    #S_nmf = nmf.fit_transform(X)
    S_nmf /= S_nmf.std() # Normalise vectors
    
    ### Plot decomposition results
    plt.figure()

    plt.subplot(2, 2, 1)
    axis_list = [pca.components_, ica.mixing_, nmf.components_]
    plot_samples_centered(X / np.std(X), labels_clust, (centering,centering), axis_list=axis_list)
    legend = plt.legend(['PCA', 'ICA', 'NMF'], loc='upper right')
    legend.set_zorder(100)
    plt.title('Observations')

    plt.subplot(2, 2, 2)
    plot_samples_centered(S_pca / np.std(S_pca, axis=0), labels_clust, (centering,centering))
    plt.title('PCA recovered signals')

    plt.subplot(2, 2, 3)
    plot_samples_centered(S_ica / np.std(S_ica), labels_clust, (centering,centering))
    plt.title('ICA recovered signals')

    plt.subplot(2, 2, 4)
    plot_samples_centered(S_nmf / np.std(S_nmf), labels_clust, (centering,centering))
    plt.title('NMF recovered signals')

    plt.show()
    
    return

In [12]:
interact_manual(Deconvolution_example,
                point_nb='10000', sigma='0.20', angle='40.0', centering='10.0',
                ica_algo=['parallel','deflation'], ica_tol='1e-03',
                ica_non_linearity=['logcosh','exp','cube'],
                nmf_init=['random','nndsvd','nndsvda','nndsvdar'], nmf_solver=['cd','mu'],
                seed='300');

interactive(children=(Text(value='10000', description='point_nb'), Text(value='0.20', description='sigma'), Te…